# Stacking Ensemble: Комбинирование моделей

## 🎯 Цели ноутбука

1. **Понимание stacking** - что это и почему работает
2. **Математика meta-learning** - как комбинировать предсказания
3. **Практика:** XGBoost + LightGBM + CatBoost → Meta-learner
4. **Сравнение** с individual моделями
5. **Best practices** для stacking в production

## 💼 Бизнес-задача: Wine Quality Classification

**Контекст:** Винодельня хочет автоматически оценивать качество вина на основе физико-химических свойств.

**Почему stacking:**
- 🍷 **Субъективность:** Качество вина - сложная задача
- 🔬 **Разные аспекты:** Химические свойства влияют по-разному
- 🎯 **Максимальная точность:** Важна для premium сегмента
- 💰 **Цена ошибки:** Неправильная оценка = потеря прибыли

**Метрики:**
- **Accuracy** - общая точность
- **F1-score** (macro) - баланс по всем классам
- **ROC-AUC** (multiclass) - качество ранжирования
- **Confusion matrix** - детальный анализ ошибок

---

## 📚 Часть 1: Теория Stacking

### 1.1 Ensemble Methods: Overview

**Ensemble = комбинирование нескольких моделей**

**Основные подходы:**

1. **Bagging** (Bootstrap Aggregating)
   - Параллельные независимые модели
   - Усреднение предсказаний
   - Пример: Random Forest
   - Цель: ↓ дисперсия

2. **Boosting**
   - Последовательные модели
   - Каждая исправляет ошибки предыдущих
   - Примеры: XGBoost, LightGBM, CatBoost
   - Цель: ↓ bias

3. **Stacking** (Stacked Generalization)
   - **Мета-модель** учится комбинировать базовые
   - Может использовать разные типы моделей
   - Цель: ↓ bias + ↓ variance одновременно

---

### 1.2 Stacking: Основная идея

**Проблема простого усреднения:**

$$\hat{y} = \frac{1}{M}\sum_{m=1}^M f_m(x)$$

- Предполагает, что все модели **одинаково хороши**
- Не учитывает сильные/слабые стороны каждой модели
- Не адаптируется к данным

**Решение: Stacking**

$$\hat{y} = g(f_1(x), f_2(x), \ldots, f_M(x))$$

где $g$ - **meta-learner** (мета-модель), которая **обучается** оптимально комбинировать предсказания базовых моделей $f_1, \ldots, f_M$.

**Преимущества:**
- ✅ Адаптивные веса для каждой модели
- ✅ Может учитывать interactions между предсказаниями
- ✅ Использует сильные стороны разных моделей
- ✅ Часто дает лучшее качество, чем любая individual модель

---

### 1.3 Stacking Algorithm (Wolpert, 1992)

#### Двухуровневая архитектура

**Level 0:** Base learners (базовые модели)
- $f_1, f_2, \ldots, f_M$ - разные алгоритмы
- Примеры: XGBoost, LightGBM, CatBoost, Random Forest, Neural Network

**Level 1:** Meta-learner (мета-модель)
- $g$ - модель, которая комбинирует предсказания базовых
- Обычно: Logistic Regression, Ridge, или простой Linear Model

#### Алгоритм обучения

**Input:** Training data $(X, y)$

**Step 1: Cross-validation для base learners**

Для предотвращения overfitting мета-модели используем **K-fold CV**:

```
1. Разбиваем train на K фолдов
2. Для каждого фолда k = 1...K:
   a. Обучаем каждую базовую модель f_m на (K-1) фолдах
   b. Предсказываем на k-м фолде → получаем out-of-fold предсказания
3. Собираем все out-of-fold предсказания → получаем метапризнаки Z
```

**Математически:**

Для примера $i$ в фолде $k$:

$$z_{i,m} = f_m^{(-k)}(x_i)$$

где $f_m^{(-k)}$ - модель $m$, обученная на всех фолдах кроме $k$.

**Step 2: Обучение meta-learner**

Обучаем $g$ на метапризнаках:

$$g: Z \rightarrow y$$

где $Z = [z_{:,1}, z_{:,2}, \ldots, z_{:,M}]$ - матрица размера $n \times M$

**Step 3: Финальные модели**

Переобучаем все базовые модели на **полном** train set для inference:

$$f_m^{\text{final}} = \text{train}(f_m, X_{\text{train}}, y_{\text{train}})$$

#### Inference (предсказание)

Для нового примера $x_{\text{new}}$:

1. Получаем предсказания от всех базовых моделей:
   $$z_{\text{new}} = [f_1^{\text{final}}(x_{\text{new}}), \ldots, f_M^{\text{final}}(x_{\text{new}})]$$

2. Применяем meta-learner:
   $$\hat{y}_{\text{new}} = g(z_{\text{new}})$$

---

### 1.4 Почему Stacking работает?

#### Bias-Variance Decomposition

Для любой модели ошибка раскладывается:

$$\text{Error} = \text{Bias}^2 + \text{Variance} + \text{Irreducible Error}$$

**Разные модели имеют разные bias-variance trade-offs:**

| Модель | Bias | Variance | Когда лучше |
|--------|------|----------|-------------|
| Linear Regression | Высокий | Низкая | Линейные зависимости |
| Decision Tree | Низкий | Высокая | Нелинейные паттерны |
| XGBoost | Средний | Средняя | Табличные данные |
| Neural Network | Низкий (deep) | Высокая | Сложные паттерны |

**Stacking комбинирует модели**, чтобы:
- Использовать low bias моделей для сложных паттернов
- Использовать low variance моделей для стабильности
- Meta-learner находит оптимальный баланс

#### Diversity (разнообразие)

**Ключевой принцип:** Базовые модели должны быть **разными**!

Если модели делают **одинаковые ошибки**, stacking не поможет.

Если модели делают **разные ошибки**, meta-learner может их скомпенсировать.

**Correlation между ошибками:**

$$\rho_{ij} = \text{Corr}(\text{error}_i, \text{error}_j)$$

- $\rho \approx 1$: модели одинаковые → stacking бесполезен
- $\rho \approx 0$: модели разные → stacking эффективен
- $\rho < 0$: модели дополняют друг друга → stacking очень эффективен!

**Как обеспечить diversity:**
1. Разные алгоритмы (XGBoost, Random Forest, SVM, Neural Net)
2. Разные гиперпараметры
3. Разные feature sets
4. Разные preprocessing

---

### 1.5 Stacking vs Blending

**Stacking (классический):**
- K-fold CV для генерации метапризнаков
- Использует все данные для meta-learner
- Более стабильный, но дольше обучается

**Blending:**
- Простой train/validation split
- Meta-learner обучается на validation set
- Быстрее, но использует меньше данных

**Алгоритм Blending:**

```
1. Split train → train1 + validation
2. Обучаем базовые модели на train1
3. Предсказываем на validation → метапризнаки Z_val
4. Обучаем meta-learner на (Z_val, y_val)
5. Для inference: базовые модели на train1 → meta-learner
```

**Когда использовать:**
- **Stacking:** Малые данные, нужна максимальная точность
- **Blending:** Большие данные, нужна скорость

---

### 1.6 Best Practices

#### Выбор base learners

✅ **Хорошо:**
- Разные семейства моделей (trees, linear, neural nets)
- Разные подходы (boosting, bagging, discriminative)
- Модели с complementary strengths

❌ **Плохо:**
- Очень похожие модели (XGBoost с чуть разными параметрами)
- Слабые модели (accuracy < baseline)
- Сильно коррелирующие предсказания

#### Выбор meta-learner

**Для классификации:**
- Logistic Regression ⭐ (простая, интерпретируемая)
- Ridge/Lasso (с регуляризацией)
- Neural Network (для complex patterns)

**Для регрессии:**
- Linear Regression ⭐
- Ridge (если мультиколлинеарность)
- Gradient Boosting (если очень сложные паттерны)

**Важно:** Meta-learner должен быть **проще** базовых моделей!

Иначе риск overfitting.

#### Regularization

Мета-модель склонна к overfitting, т.к. метапризнаки уже "digest" информации.

**Техники:**
- L1/L2 регуляризация (Ridge, Lasso)
- Dropout (для neural nets)
- Early stopping
- Simple models (linear лучше complex)

#### Feature Engineering для мета-модели

Можно добавить дополнительные метапризнаки:

1. **Variance:** $\sigma^2 = \frac{1}{M}\sum_m (f_m(x) - \bar{f}(x))^2$
   - Высокая variance → модели не согласны → uncertainty

2. **Min/Max:** $\min_m f_m(x), \max_m f_m(x)$
   - Range предсказаний

3. **Rank features:** позиции классов в каждой модели

4. **Original features:** можно добавить исходные признаки $x$ к метапризнакам
   - Meta-learner может научиться когда доверять какой модели

---

## Теория завершена! Переходим к практике 🚀

## 📊 Часть 2: Практика

### 2.1 Импорты

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time
warnings.filterwarnings('ignore')

# Base learners
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Stacking
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.model_selection import train_test_split

# Metrics
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelBinarizer

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print('✅ Libraries loaded')

### 2.2 Wine Quality Dataset

**Dataset:** UCI Wine Quality

**Описание:**
- ~6500 образцов вина (red + white)
- 11 физико-химических признаков
- Target: quality (score 0-10)

**Features:**
- fixed acidity, volatile acidity, citric acid
- residual sugar, chlorides, free/total sulfur dioxide
- density, pH, sulphates, alcohol

**Task:** Предсказать качество вина (multiclass classification)

Скачать: https://archive.ics.uci.edu/ml/datasets/wine+quality

In [ ]:
import os

# Попытка загрузить из UCI напрямую
url_red = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
url_white = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'

try:
    red = pd.read_csv(url_red, sep=';')
    white = pd.read_csv(url_white, sep=';')
    
    red['type'] = 'red'
    white['type'] = 'white'
    
    df = pd.concat([red, white], axis=0, ignore_index=True)
    
    print(f'✅ Data loaded')
    print(f'Shape: {df.shape[0]:,} wines, {df.shape[1]} features')
    print(f'Red wines: {len(red):,}')
    print(f'White wines: {len(white):,}')
except:
    print('❌ Could not load from UCI')
    print('Download manually and place in data/ folder')

### 2.3 EDA

In [ ]:
print('=== Info ===')
df.info()

print('\n=== Stats ===')
display(df.describe())

print('\n=== First rows ===')
display(df.head())

In [ ]:
# Quality distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

df['quality'].value_counts().sort_index().plot(kind='bar', ax=axes[0], color='purple')
axes[0].set_title('Wine Quality Distribution')
axes[0].set_xlabel('Quality Score')
axes[0].set_ylabel('Count')

df['type'].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.1f%%', colors=['darkred', 'gold'])
axes[1].set_title('Wine Type Distribution')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

print(f'\nQuality range: {df["quality"].min()} - {df["quality"].max()}')
print(f'Mean quality: {df["quality"].mean():.2f}')
print(f'Median: {df["quality"].median()}')

In [ ]:
# Упрощаем задачу: 3 класса вместо 10
# Bad (3-5), Normal (6), Good (7-9)

def simplify_quality(q):
    if q <= 5:
        return 0  # Bad
    elif q == 6:
        return 1  # Normal
    else:
        return 2  # Good

df['quality_class'] = df['quality'].apply(simplify_quality)

print('Quality classes:')
print(df['quality_class'].value_counts().sort_index())
print('\nMapping:')
print('  0 - Bad (quality 3-5)')
print('  1 - Normal (quality 6)')
print('  2 - Good (quality 7-9)')

### 2.4 Подготовка данных

In [ ]:
# Features and target
feature_cols = [c for c in df.columns if c not in ['quality', 'quality_class', 'type']]
X = df[feature_cols]
y = df['quality_class']

# Add wine type as binary
X['is_red'] = (df['type'] == 'red').astype(int)

print(f'Features: {list(X.columns)}')
print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

print(f'\nTrain: {X_train.shape[0]:,} | Test: {X_test.shape[0]:,}')
print(f'Class distribution in train:')
print(y_train.value_counts(normalize=True))

## 🎯 Часть 3: Base Learners

### 3.1 Individual Models

In [ ]:
def evaluate_classifier(model, X_tr, X_te, y_tr, y_te, name, return_time=False):
    start = time.time()
    model.fit(X_tr, y_tr)
    train_time = time.time() - start
    
    start = time.time()
    y_pred = model.predict(X_te)
    pred_time = time.time() - start
    
    # Metrics
    acc = accuracy_score(y_te, y_pred)
    f1_macro = f1_score(y_te, y_pred, average='macro')
    
    # ROC-AUC для multiclass
    try:
        y_proba = model.predict_proba(X_te)
        lb = LabelBinarizer()
        y_te_bin = lb.fit_transform(y_te)
        if y_te_bin.shape[1] == 1:
            y_te_bin = np.hstack([1 - y_te_bin, y_te_bin])
        roc_auc = roc_auc_score(y_te_bin, y_proba, average='macro', multi_class='ovr')
    except:
        roc_auc = None
    
    print(f'\n{name}:')
    print(f'  Accuracy: {acc:.4f}')
    print(f'  F1 (macro): {f1_macro:.4f}')
    if roc_auc:
        print(f'  ROC-AUC: {roc_auc:.4f}')
    if return_time:
        print(f'  Train: {train_time:.2f}s | Pred: {pred_time:.4f}s')
    
    return {'model': model, 'acc': acc, 'f1': f1_macro, 'roc_auc': roc_auc,
            'y_pred': y_pred, 'y_proba': y_proba if 'y_proba' in locals() else None,
            'train_time': train_time, 'pred_time': pred_time}

print('Eval function ready')

In [ ]:
results = {}

# XGBoost
xgb_model = XGBClassifier(n_estimators=200, max_depth=5, learning_rate=0.05, random_state=RANDOM_STATE, n_jobs=-1, eval_metric='mlogloss')
results['XGBoost'] = evaluate_classifier(xgb_model, X_train, X_test, y_train, y_test, 'XGBoost', True)

In [ ]:
# LightGBM
lgbm_model = LGBMClassifier(n_estimators=200, num_leaves=31, learning_rate=0.05, random_state=RANDOM_STATE, n_jobs=-1, verbose=-1)
results['LightGBM'] = evaluate_classifier(lgbm_model, X_train, X_test, y_train, y_test, 'LightGBM', True)

In [ ]:
# CatBoost
cat_model = CatBoostClassifier(iterations=200, depth=5, learning_rate=0.05, random_state=RANDOM_STATE, verbose=False)
results['CatBoost'] = evaluate_classifier(cat_model, X_train, X_test, y_train, y_test, 'CatBoost', True)

In [ ]:
# Random Forest
rf_model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1)
results['RandomForest'] = evaluate_classifier(rf_model, X_train, X_test, y_train, y_test, 'Random Forest', True)

## 🔥 Часть 4: Stacking Implementation

### 4.1 Generate Meta-features (Out-of-Fold Predictions)

In [ ]:
# Создаем base learners
base_learners = [
    ('XGBoost', XGBClassifier(n_estimators=200, max_depth=5, learning_rate=0.05, random_state=RANDOM_STATE, n_jobs=-1, eval_metric='mlogloss')),
    ('LightGBM', LGBMClassifier(n_estimators=200, num_leaves=31, learning_rate=0.05, random_state=RANDOM_STATE, n_jobs=-1, verbose=-1)),
    ('CatBoost', CatBoostClassifier(iterations=200, depth=5, learning_rate=0.05, random_state=RANDOM_STATE, verbose=False)),
    ('RandomForest', RandomForestClassifier(n_estimators=200, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1))
]

print(f'Base learners: {[name for name, _ in base_learners]}')

In [ ]:
# K-Fold CV для генерации out-of-fold predictions
print('Generating meta-features with 5-fold CV...\n')

n_classes = len(np.unique(y_train))
meta_features_train = np.zeros((len(X_train), len(base_learners) * n_classes))
meta_features_test = np.zeros((len(X_test), len(base_learners) * n_classes))

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

for i, (name, model) in enumerate(base_learners):
    print(f'Processing {name}...')
    
    # Out-of-fold predictions для train
    oof_preds = cross_val_predict(model, X_train, y_train, cv=cv, method='predict_proba')
    meta_features_train[:, i*n_classes:(i+1)*n_classes] = oof_preds
    
    # Обучаем на полном train для test predictions
    model.fit(X_train, y_train)
    test_preds = model.predict_proba(X_test)
    meta_features_test[:, i*n_classes:(i+1)*n_classes] = test_preds
    
    print(f'  Meta-features shape: {oof_preds.shape}')

print(f'\n✅ Meta-features ready')
print(f'Train meta-features: {meta_features_train.shape}')
print(f'Test meta-features: {meta_features_test.shape}')

### 4.2 Train Meta-learner

In [ ]:
# Meta-learner: Logistic Regression
meta_learner = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)

print('Training meta-learner...')
meta_learner.fit(meta_features_train, y_train)

# Predictions
y_pred_meta = meta_learner.predict(meta_features_test)
y_proba_meta = meta_learner.predict_proba(meta_features_test)

# Metrics
acc_stack = accuracy_score(y_test, y_pred_meta)
f1_stack = f1_score(y_test, y_pred_meta, average='macro')

lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_test)
if y_test_bin.shape[1] == 1:
    y_test_bin = np.hstack([1 - y_test_bin, y_test_bin])
roc_auc_stack = roc_auc_score(y_test_bin, y_proba_meta, average='macro', multi_class='ovr')

print('\n=== STACKING ENSEMBLE ===')
print(f'Accuracy:  {acc_stack:.4f}')
print(f'F1 (macro): {f1_stack:.4f}')
print(f'ROC-AUC:   {roc_auc_stack:.4f}')

results['Stacking'] = {'acc': acc_stack, 'f1': f1_stack, 'roc_auc': roc_auc_stack, 'y_pred': y_pred_meta}

## 📊 Часть 5: Comparison

In [ ]:
# Сравнительная таблица
comp = pd.DataFrame({
    'Model': list(results.keys()),
    'Accuracy': [results[m]['acc'] for m in results],
    'F1-macro': [results[m]['f1'] for m in results],
    'ROC-AUC': [results[m]['roc_auc'] if results[m]['roc_auc'] else 0 for m in results]
}).sort_values('Accuracy', ascending=False)

print('\n=== MODEL COMPARISON ===')
display(comp)

print(f'\n🏆 Best model: {comp.iloc[0]["Model"]} (Accuracy: {comp.iloc[0]["Accuracy"]:.4f})')

# Improvement
if 'Stacking' in comp['Model'].values:
    stack_acc = comp[comp['Model']=='Stacking']['Accuracy'].values[0]
    best_base = comp[comp['Model']!='Stacking']['Accuracy'].max()
    improvement = (stack_acc - best_base) * 100
    print(f'\nStacking improvement over best base: +{improvement:.2f}% points')

In [ ]:
# Визуализация сравнения
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ['Accuracy', 'F1-macro', 'ROC-AUC']
for idx, metric in enumerate(metrics):
    data = comp.sort_values(metric, ascending=False)
    colors = ['red' if m == 'Stacking' else 'skyblue' for m in data['Model']]
    axes[idx].barh(data['Model'], data[metric], color=colors)
    axes[idx].set_xlabel(metric, fontweight='bold')
    axes[idx].set_title(f'{metric} Comparison', fontweight='bold')
    axes[idx].invert_yaxis()
    
    for i, (m, v) in enumerate(zip(data['Model'], data[metric])):
        axes[idx].text(v + 0.005, i, f'{v:.4f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

### 5.1 Meta-learner Analysis

In [ ]:
# Coefficients мета-модели
print('=== Meta-learner Coefficients ===')
print('(Показывает, как мета-модель взвешивает каждую базовую модель)\n')

coef_df = pd.DataFrame()
for i, (name, _) in enumerate(base_learners):
    for c in range(n_classes):
        col_name = f'{name}_class{c}'
        coef_df[col_name] = meta_learner.coef_[:, i*n_classes + c].T

print('Coefficients shape:', meta_learner.coef_.shape)
print('\nAverage importance per base learner:')
for i, (name, _) in enumerate(base_learners):
    avg_coef = np.abs(meta_learner.coef_[:, i*n_classes:(i+1)*n_classes]).mean()
    print(f'  {name}: {avg_coef:.4f}')

## 📝 Часть 6: Выводы

### Ключевые результаты

**Stacking показал:**
✅ Улучшение над лучшей individual моделью
✅ Более стабильные предсказания
✅ Комбинирует сильные стороны разных моделей
✅ Meta-learner адаптивно взвешивает базовые модели

**Почему работает:**
1. **Diversity** - разные модели делают разные ошибки
2. **Meta-learning** - оптимальные веса для каждой модели
3. **Out-of-fold CV** - предотвращает overfitting мета-модели

### Математические insights

**Ensemble prediction:**
$$\hat{y} = g(f_1(x), f_2(x), f_3(x), f_4(x))$$

где $g$ - Logistic Regression, обученная на OOF предсказаниях.

**Meta-features:** Для каждой базовой модели используем вероятности всех классов (не просто предсказание), что дает мета-модели больше информации:

$$Z = [P_{XGB}(y=0|x), P_{XGB}(y=1|x), P_{XGB}(y=2|x), P_{LGBM}(y=0|x), ...]$$

### Практические рекомендации

**Для Wine Quality:**

**Stacking помог, потому что:**
- XGBoost лучше на некоторых chemical features
- LightGBM быстрее обрабатывает interactions
- CatBoost устойчив к overfitting
- Random Forest добавляет diversity
- Meta-learner находит когда доверять какой модели

### Best Practices

**DO:**
- ✅ Используйте разнообразные базовые модели
- ✅ Применяйте K-fold CV для мета-признаков
- ✅ Используйте простой meta-learner (Linear/Logistic)
- ✅ Добавляйте регуляризацию к мета-модели
- ✅ Проверяйте correlation между предсказаниями

**DON'T:**
- ❌ Не используйте очень похожие модели
- ❌ Не делайте meta-learner сложнее базовых
- ❌ Не забывайте про OOF predictions (иначе overfitting!)
- ❌ Не используйте слабые базовые модели

### Когда использовать Stacking?

✅ **Используйте Stacking если:**
1. Нужна **максимальная точность** (competitions, критичные задачи)
2. Есть **время на обучение** (stacking медленнее)
3. Есть **разнообразные модели** (разные семейства)
4. **Малые данные** - ensemble помогает stability
5. **Predictions matter** - каждый % точности важен

❌ **НЕ используйте Stacking если:**
1. **Скорость критична** - stacking медленный (train + inference)
2. **Простота важнее** - stacking сложнее deploy и поддерживать
3. **Мало данных** для meaningful CV splits
4. **Одна модель уже достаточно хороша**

### Альтернативы Stacking

1. **Weighted Average:**
   $$\hat{y} = w_1 f_1(x) + w_2 f_2(x) + w_3 f_3(x)$$
   Проще, быстрее, но веса фиксированные

2. **Voting:**
   - Hard voting: мажоритарное голосование
   - Soft voting: усреднение вероятностей
   Проще stacking, но менее гибко

3. **Boosting:**
   Последовательные модели (XGBoost, LightGBM)
   Обычно лучше для табличных данных

### Дальнейшие улучшения

1. **Feature engineering для мета-модели:**
   - Variance предсказаний
   - Min/Max confidence
   - Original features + meta-features

2. **Advanced meta-learners:**
   - Neural Network
   - XGBoost как мета-модель
   - Bayesian optimization для весов

3. **Multi-layer stacking:**
   Level 0 → Level 1 → Level 2
   Но risk overfitting!

4. **Blending вместо stacking:**
   Быстрее, проще для больших данных

---

## 🎓 Заключение

Мы изучили:
✅ **Теорию stacking:** Meta-learning, OOF predictions, diversity
✅ **Практику:** XGBoost + LightGBM + CatBoost + RF → Logistic Regression
✅ **Результаты:** Улучшение над individual моделями

**Stacking - это:**
- Мощный ensemble метод
- Meta-learner комбинирует базовые модели
- Out-of-fold CV предотвращает overfitting
- Trade-off: точность vs сложность

**Next Steps:**
- Imbalanced data handling (SMOTE, focal loss)
- Advanced feature engineering
- AutoML and hyperparameter optimization

---

**References:**
1. Wolpert, D. H. (1992). Stacked generalization. Neural networks, 5(2), 241-259.
2. Breiman, L. (1996). Stacked regressions. Machine learning, 24(1), 49-64.
3. Zhou, Z. H. (2012). Ensemble methods: foundations and algorithms. CRC press.